In [1]:
!pip install telepot
!pip install geocoder
!pip install firebase-admin

In [2]:
!pip install --upgrade requests urllib3 telepot

  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/63.9 kB ? eta -:--:--
   ---------------------------------------- 63.9/63.9 kB 3.6 MB/s eta 0:00:00
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.165 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
pyppeteer 1.0.2 requires urllib3<2.0.0,>=1.25.8, but you have urllib3 2.2.1 which is incompatible.
pyrebase 3.0.27 requires requests==2.11.1, but you have requests 2.32.2 which is incompatible.


In [3]:
import numpy as np
import cv2
from keras.models import load_model
from collections import deque
import telepot
import time
import geocoder
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [4]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate("./realtime.json")  # Replace with your service account key path
firebase_admin.initialize_app(cred)
db = firestore.client()
collection_ref = db.collection('your_collection_name')


In [6]:
def get_current_location():
    # Get current location
    location = geocoder.ip('me')
    location_info = f"LOCATION: {location.city}, {location.state}, {location.country}\n"
    return location_info

def save_violence_alert(location_info, timeMoment, filename, video_filename):
    # Save violence alert data to Firestore
    doc_ref = db.collection(u'violence_alerts').document()
    doc_ref.set({
        u'location_info': location_info,
        u'time': timeMoment,
        u'filename': filename,
        u'video_filename': video_filename
    })
    print("Violence alert data saved to Firestore")

def send_telegram_message(bot, chat_id, message, video_path, photo_path):
    try:
        bot.sendMessage(chat_id, message)
        with open(video_path, 'rb') as video_file:
            bot.sendVideo(chat_id, video_file)
        with open(photo_path, 'rb') as photo_file:
            bot.sendPhoto(chat_id, photo_file)
    except Exception as e:
        print(f"Error sending message: {e}")

def print_results(video):
    # Initializing variables
    trueCount = 0
    imageSaved = 0
    filename = 'savedImage_detect.jpg'
    video_filename = 'recordedVideo.avi'
    sendAlert = 0
    
    # Get current location
    location_info = get_current_location()
    
    # Loading pre-trained Keras model
    print("Loading model ...")
    model = load_model('./modelnew.h5')
    Q = deque(maxlen=128)
    
    # Capturing video
    vs = cv2.VideoCapture(video)
    writer = None
    (W, H) = (None, None)
    count = 0
    
    # Looping through video frames
    while True:
        (grabbed, frame) = vs.read()
        
        if not grabbed:
            break
        
        if W is None or H is None:
            (H, W) = frame.shape[:2]
        
        output = frame.copy()
        
        # Preprocessing frame for model input
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255
        
        # Predicting violence probability
        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)
        
        # Calculating average prediction
        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]  # Thresholding predictions
        label = i
        
        # Determining text color based on violence label
        text_color = (0, 255, 0)  # default: green
        if label:  # Violence prob
            text_color = (0, 0, 255)  # red
            trueCount += 1
        
        # Adding text to the output frame
        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)
        
        # Initializing video writer if not done
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(video_filename, fourcc, 30, (W, H), True)
        
        # Writing frame to output video
        writer.write(output)
        
        # Displaying output frame
        cv2.imshow("Output", output)
        
        # Saving image and sending alert if violence detected for 50 consecutive frames
        if trueCount == 50:
            if imageSaved == 0:
                if label:
                    cv2.imwrite(filename, output)  # Save the frame
                    imageSaved = 1
            
            if sendAlert == 0:
                timeMoment = time.strftime("%H:%M:%S \nDate %d-%m-%y")
                location_info = get_current_location()
                
                # Save violence alert data to Firestore
                save_violence_alert(location_info, timeMoment, filename, video_filename)
                
                # Sending alert through Telegram
                bot = telepot.Bot('7186069023:AAFg_KT_mSUd5ofySXlwViyK4k32Gu3ahnU')
                chat_id = '-1001991802221'
                message = f"VIOLENCE ALERT!!\n{location_info}TIME: {timeMoment}"
                
                send_telegram_message(bot, chat_id, message, video_filename, filename)
                sendAlert = 1
        
        # Checking for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
    
    # Cleaning up
    print("[INFO] cleaning up...")
    writer.release()
    vs.release()

V_path = "./New Dataset/Fight-20240507T205137Z-001/Fight/F3.4.mp4"
print_results(V_path)


Loading model ...


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━

Error sending message: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot7186069023:AAFg_KT_mSUd5ofySXlwViyK4k32Gu3ahnU/sendVideo (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2406)')))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━